<a href="https://colab.research.google.com/github/myousefi/google-colab-notebooks/blob/main/sleep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the Excel file in your Google Drive
file_path = '/content/drive/MyDrive/PPG/Sleep/Egr2_M4_3447_Sleep.xlsx'  # Adjust this to your file's actual path

# Specify the sheet name you want to process
sheet_name = 'RA Sal'

# Read the specified sheet, skipping the first two rows
df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.set_index(["Sequence", "CO2onset", "Measure"], inplace=True, drop=False)

result_dict = {}

for seq, seq_df in df.groupby(level="Sequence"):
    co2_onset = seq_df.index.get_level_values(1).unique().values[0]

    peaks_df = seq_df.loc[(seq, slice(None), ["PeakTimes", "Peaks"]), :]

    troughs_df = seq_df.loc[(seq, slice(None), ["TroughTimes", "Troughs"]), :]

    peaks_series = pd.Series(data=peaks_df.iloc[1, 4:].values, index=peaks_df.iloc[0, 4:].values - co2_onset).dropna()

    troughs_series = pd.Series(data=troughs_df.iloc[1, 4:].values, index=troughs_df.iloc[0, 4:].values - co2_onset).dropna()

    result_dict[seq] = (peaks_series, troughs_series)

In [ ]:
# Assuming co2_onset_list is defined as follows (example values):
co2_onset_list = [2558, 3878, 5198]

# Extract unique CO2onset values and their corresponding sequences from the DataFrame
unique_co2_onsets = df.index.get_level_values("CO2onset").unique()
sequences = df.index.get_level_values("Sequence").unique()

# Initialize a dictionary to hold the closest sequence for each CO2onset value in co2_onset_list
closest_sequences = {}

# Loop through each value in co2_onset_list to find the closest CO2onset in the DataFrame
for target_co2_onset in co2_onset_list:
    # Calculate the absolute difference between the target CO2onset and all CO2onset values in the DataFrame
    differences = abs(unique_co2_onsets - target_co2_onset)

    # Find the index of the minimum difference
    min_diff_index = differences.argmin()

    # Use the index to find the corresponding sequence
    closest_sequence = sequences[min_diff_index]

    # Store the result in the dictionary
    closest_sequences[target_co2_onset] = closest_sequence

In [ ]:
# prompt: set simple_white as plotly theme at pio leve

import plotly.io as pio

pio.templates.default = "simple_white"


In [ ]:
import plotly.graph_objects as go

# Create a figure for Peaks
fig_peaks = go.Figure()
fig_troughs = go.Figure()

for onset, seq in closest_sequences.items():
    peaks_series, troughs_series = result_dict[seq][0], result_dict[seq][1]
    fig_peaks.add_trace(
        go.Scatter(
            x=peaks_series.index,
            y=peaks_series.values,
            mode="lines+markers",
            name=onset,
        )
    )
    fig_peaks.update_layout(
        title="Peaks Plot", xaxis_title="Index", yaxis_title="Value"
    )

    fig_troughs.add_trace(
        go.Scatter(
            x=troughs_series.index,
            y=troughs_series.values,
            mode="lines+markers",
            name=onset,
        )
    )

fig_troughs.update_layout(
    title="Troughs Plot", xaxis_title="Index", yaxis_title="Value"
)

# Display the figures
fig_peaks.show()
fig_troughs.show()